In [1]:
#Import libraries

#Import generally useful packages
import numpy as np
import os
import scipy.cluster
from scipy import stats
import re

#Import plotting packages
import seaborn as sns
import bokeh.io
import bokeh.plotting
import bokeh.transform

#Import bio specific packages
import Bio
from Bio import SeqIO
from Bio.Seq import Seq
from Bio import pairwise2
from Bio import Align
from scipy.stats import norm

bokeh.io.output_notebook()

Loading BokehJS ...

In [2]:
#Stores data into list of lists alongside file names

#Set working directory
workingdir = os.getcwd()

#Initialize lists for data and file names
data = []
names = []
#Iterate through files, store data in lists
for file in os.listdir(workingdir):
    if file.endswith(".fasta"):
        filedata = []
        #Append file name to list
        names.append(file.split('.')[0].upper())
        filedata.append(file.split('.')[0].upper())
        with open(file, 'r') as f:
            #Append reads as list to large list
            seqs = f.read().replace('-', '').splitlines() #removes dashes here
            filedata.append(seqs[1::2])
        data.append(filedata)
            
data = sorted(data, key=lambda file: file[0])
names = sorted(names)

In [3]:
print(names)

['RUN144_289_WT_DIV', 'RUN144_295_NOHAIRPIN', 'RUN144_296_DEL3HOM_DIV', 'RUN144_298_DELSPACER_DIV', 'RUN144_306_DEL5HOM_DIV', 'RUN156_205_WT_DIV', 'RUN178_322_J902NOINTRON', 'RUN178_323_J902NOINTRON', 'RUN199_AVD105A', 'RUN199_AVD4A', 'RUN199_AVD54A', 'RUN199_AVD55A', 'RUN199_AVD57A']


In [4]:
#Helper Functions

Div = "TTCCGTGGTGGCTACTGGAGCGACGCCTCGGATGCGGGGTTGTCGGCGTTGCGCTTGTACAACGCCCGCGGCGATCGCAGCAGCTTCA"
Div2 = "TGGCTACTGGAGCGACGCCTCGGATGCGGGGTTGTCGGCGTTGCGCTTGTACAACGCCCGCGGCGATCGCAGCAGCTTCA"

def start(read, divstring):
    for i in range(len(read)):
        if read[i] != divstring[i]:
            if read[i] != "N":
                return i+1
    else:
        return 0
    
def end(read, divstring):
    maxlen = len(read)
    for i in range(maxlen):
        if read[-i-1] != divstring[-i-1]:
            if read[-i-1] != "N":
                return maxlen-i
    else:
        return 0
    
def startendzeros(readlist, divstring):
    x = []
    y = []
    for read in readlist:
        x.append(start(read, divstring))
        y.append(end(read, divstring))
    return x, y

def startend(readlist, divstring):
    x = []
    y = []
    for read in readlist:
        startpos = start(read, divstring)
        endpos = end(read, divstring)
        if (startpos != 0) or (endpos != 0):
            x.append(startpos)
            y.append(endpos)
    return x, y

def startendoffdiag(readlist, divstring):
    x = []
    y = []
    for read in readlist:
        startpos = start(read, divstring)
        endpos = end(read, divstring)
        if (startpos != 0) or (endpos != 0):
            if (startpos != endpos):
                x.append(startpos)
                y.append(endpos)
    return x, y

def startend_thresh(readlist, divstring, thresh):
    x = []
    y = []
    for read in readlist:
        startpos = start(read, divstring)
        endpos = end(read, divstring)
        if (endpos >= thresh) and (startpos != endpos):
            x.append(startpos)
            y.append(endpos)
    return x, y

def startend_leftthresh(readlist, divstring, thresh):
    x = []
    y = []
    for read in readlist:
        startpos = start(read, divstring)
        endpos = end(read, divstring)
        if (startpos <= thresh) and (startpos != endpos):
            x.append(startpos)
            y.append(endpos)
    return x, y

def startend_thresh_plusdiag(readlist, divstring, thresh):
    x = []
    y = []
    for read in readlist:
        startpos = start(read, divstring)
        endpos = end(read, divstring)
        if (endpos >= thresh) and (startpos != 0):
            x.append(startpos)
            y.append(endpos)
    return x, y

def startend_leftthresh_plusdiag(readlist, divstring, thresh):
    x = []
    y = []
    for read in readlist:
        startpos = start(read, divstring)
        endpos = end(read, divstring)
        if (startpos <= thresh) and (startpos != 0):
            x.append(startpos)
            y.append(endpos)
    return x, y

In [5]:
#Main Plotting Function

def plotstartend(x, y, divstring):
    binnum = len(divstring)
    
    #Plot scatter plot
    p = bokeh.plotting.figure(x_axis_label="5' Div Start", y_axis_label="3' End Div", plot_width=400, plot_height=400,
                             x_range =(-0.1*binnum, binnum*1.1), y_range =(-0.1*binnum, binnum*1.1))
    p.scatter(x, y, alpha = 0.2)

    #Calculate x histogram
    hhist, hedges = np.histogram(x, bins=binnum)
    hzeros = np.zeros(len(hedges)-1)
    hmax = max(hhist)*1.1

    #Plot x histogram
    ph = bokeh.plotting.figure(toolbar_location=None, plot_width=p.plot_width, plot_height=150, x_range=p.x_range,
                y_range=(0, hmax), min_border=10, min_border_left=50, y_axis_location="right")

    ph.quad(bottom=0, left=hedges[:-1], right=hedges[1:], top=hhist, color="white", line_color="#3A5785")
    hh1 = ph.quad(bottom=0, left=hedges[:-1], right=hedges[1:], top=hzeros, alpha=0.5)
    hh2 = ph.quad(bottom=0, left=hedges[:-1], right=hedges[1:], top=hzeros, alpha=0.1)

    #Calculate y histogram
    vhist, vedges = np.histogram(y, bins=binnum)
    vzeros = np.zeros(len(vedges)-1)
    vmax = max(vhist)*1.1

    #Plot y histogram
    pv = bokeh.plotting.figure(toolbar_location=None, plot_width=150, plot_height=p.plot_height, x_range=(0, vmax),
                y_range=p.y_range, min_border=10, y_axis_location="right")
    pv.quad(left=0, bottom=vedges[:-1], top=vedges[1:], right=vhist, color="white", line_color="#3A5785")
    vh1 = pv.quad(left=0, bottom=vedges[:-1], top=vedges[1:], right=vzeros, alpha=0.5)
    vh2 = pv.quad(left=0, bottom=vedges[:-1], top=vedges[1:], right=vzeros, alpha=0.1)

    #Plot overall figure
    finallayout = bokeh.plotting.gridplot([[p, pv], [ph, None]], merge_tools=False)
    bokeh.io.show(finallayout)

## Plots for J900 Plasmids

In [6]:
#Plot for WT Rep 1
print(data[5][0])
#x, y = startendzeros(data[5][1])
#x, y = startend(data[5][1])
x, y = startendoffdiag(data[5][1], Div)
#x, y = startend_thresh(data[5][1], Div, 80)
#x, y = startend_leftthresh(data[5][1], Div, 15)
plotstartend(x, y, Div)

RUN156_205_WT_DIV


In [7]:
#Plot for WT Rep 2
print(data[0][0])
x, y = startendoffdiag(data[0][1], Div)
plotstartend(x, y, Div)

RUN144_289_WT_DIV


In [8]:
#Plot for 5' homology deletion
print(data[4][0])
x, y = startendoffdiag(data[4][1], Div)
plotstartend(x, y, Div)

RUN144_306_DEL5HOM_DIV


In [9]:
#Plot for spacer deletion
print(data[3][0])
x, y = startendoffdiag(data[3][1], Div)
plotstartend(x, y, Div)

RUN144_298_DELSPACER_DIV


In [10]:
#Plot for 3' homology deletion
print(data[2][0])
x, y = startendoffdiag(data[2][1], Div)
plotstartend(x, y, Div)

RUN144_296_DEL3HOM_DIV


In [11]:
#Plot for Hairpin deletion
print(data[1][0])
x, y = startendoffdiag(data[1][1], Div)
plotstartend(x, y, Div)

RUN144_295_NOHAIRPIN


## Plots for Avd mutations

In [12]:
#Plot for Avd 105A deletion
print(data[8][0])
x, y = startendoffdiag(data[8][1], Div)
plotstartend(x, y, Div)

RUN199_AVD105A


In [13]:
#Plot for Avd 4A deletion
print(data[9][0])
x, y = startendoffdiag(data[9][1], Div)
plotstartend(x, y, Div)

RUN199_AVD4A


In [14]:
#Plot for Avd 54A deletion
print(data[10][0])
x, y = startendoffdiag(data[10][1], Div)
plotstartend(x, y, Div)

RUN199_AVD54A


In [15]:
#Plot for Avd 55A deletion
print(data[11][0])
x, y = startendoffdiag(data[11][1], Div)
plotstartend(x, y, Div)

RUN199_AVD55A


In [16]:
#Plot for Avd 57A deletion
print(data[12][0])
x, y = startendoffdiag(data[12][1], Div)
plotstartend(x, y, Div)

RUN199_AVD57A


## Comparison to J902 (minus intron, truncated left end)

In [17]:
#Generate artificial truncations:

WTtruncate1 = []
for read in data[5][1]:
    WTtruncate1.append(read[8:])

WTtruncate2 = []
for read in data[0][1]:
    WTtruncate2.append(read[8:])
    
del3homtruncate = []
for read in data[2][1]:
    del3homtruncate.append(read[8:])
    
delspacertruncate = []
for read in data[3][1]:
    delspacertruncate.append(read[8:])

del5homtruncate = []
for read in data[4][1]:
    del5homtruncate.append(read[8:])

In [18]:
#Plot for J902 WT Rep 1
print(data[6][0])
x, y = startendoffdiag(data[6][1], Div2)
#x, y = startend(data[6][1], Div2)
plotstartend(x, y, Div2)

RUN178_322_J902NOINTRON


In [19]:
#Plot for J902 WT Rep 2
print(data[7][0])
x, y = startendoffdiag(data[7][1], Div2)
#x, y = startend(data[7][1], Div2)
plotstartend(x, y, Div2)

RUN178_323_J902NOINTRON


In [20]:
#Plot for artificially truncated J900 WT Rep 1
print(data[5][0])
x, y = startendoffdiag(WTtruncate1, Div2)
#x, y = startend(WTtruncate1, Div2)
plotstartend(x, y, Div2)

RUN156_205_WT_DIV


In [21]:
#Plot for artificially truncated J900 WT Rep 2
print(data[0][0])
x, y = startendoffdiag(WTtruncate2, Div2)
#x, y = startend(WTtruncate2, Div2)
plotstartend(x, y, Div2)

RUN144_289_WT_DIV


In [22]:
#Plot for artificially truncated 5' homology deletion
print(data[4][0])
x, y = startendoffdiag(del5homtruncate, Div2)
#x, y = startend(del5homtruncate, Div2)
plotstartend(x, y, Div2)

RUN144_306_DEL5HOM_DIV


In [23]:
#Plot for artificially truncated spacer deletion
print(data[3][0])
x, y = startendoffdiag(delspacertruncate, Div2)
#x, y = startend(delspacertruncate, Div2)
plotstartend(x, y, Div2)

RUN144_298_DELSPACER_DIV


In [24]:
#Plot for artificially truncated 3' homology deletion
print(data[2][0])
x, y = startendoffdiag(del3homtruncate, Div2)
#x, y = startend(del3homtruncate, Div2)
plotstartend(x, y, Div2)

RUN144_296_DEL3HOM_DIV


## Analysis of 5' Position for Fully Diversified 3' Ends

In [25]:
#Plot for J902 WT Rep 1, thresholded
print(data[6][0])
x, y = startend_thresh(data[6][1], Div2, 70)
plotstartend(x, y, Div2)

RUN178_322_J902NOINTRON


In [26]:
#Plot for J902 WT Rep 2, thresholded
print(data[7][0])
x, y = startend_thresh(data[7][1], Div2, 70)
plotstartend(x, y, Div2)

RUN178_323_J902NOINTRON


In [27]:
#Plot for artificially truncated J900 WT Rep 1, thresholded
print(data[5][0])
x, y = startend_thresh(WTtruncate1, Div2, 70)
plotstartend(x, y, Div2)

RUN156_205_WT_DIV


In [28]:
#Plot for artificially truncated J900 WT Rep 2, thresholded
print(data[0][0])
x, y = startend_thresh(WTtruncate2, Div2, 70)
plotstartend(x, y, Div2)

RUN144_289_WT_DIV


In [29]:
#Plot for artificially truncated 5' homology deletion, thresholded
print(data[4][0])
x, y = startend_thresh(del5homtruncate, Div2, 70)
plotstartend(x, y, Div2)

RUN144_306_DEL5HOM_DIV


In [30]:
#Plot for artificially truncated spacer deletion, thresholded
print(data[3][0])
x, y = startend_thresh(delspacertruncate, Div2, 70)
plotstartend(x, y, Div2)

RUN144_298_DELSPACER_DIV


In [31]:
#Plot for artificially truncated 3' homology deletion, thresholded
print(data[2][0])
x, y = startend_thresh(del3homtruncate, Div2, 70)
plotstartend(x, y, Div2)

RUN144_296_DEL3HOM_DIV


## Analysis of 3' Position for Fully Diversified 5' Ends

In [32]:
#Plot for J902 WT Rep 1, left thresholded
print(data[6][0])
x, y = startend_leftthresh(data[6][1], Div2, 10)
plotstartend(x, y, Div2)

RUN178_322_J902NOINTRON


In [33]:
#Plot for J902 WT Rep 2, left thresholded
print(data[7][0])
x, y = startend_leftthresh(data[7][1], Div2, 10)
plotstartend(x, y, Div2)

RUN178_323_J902NOINTRON


In [34]:
#Plot for artificially truncated J900 WT Rep 1, left thresholded
print(data[0][0])
x, y = startend_leftthresh(WTtruncate1, Div2, 10)
plotstartend(x, y, Div2)

RUN144_289_WT_DIV


In [35]:
#Plot for artificially truncated J900 WT Rep 2, left thresholded
print(data[5][0])
x, y = startend_leftthresh(WTtruncate2, Div2, 10)
plotstartend(x, y, Div2)

RUN156_205_WT_DIV


In [36]:
#Plot for artificially truncated 5' homology deletion, left thresholded
print(data[4][0])
x, y = startend_leftthresh(del5homtruncate, Div2, 10)
plotstartend(x, y, Div2)

RUN144_306_DEL5HOM_DIV


In [37]:
#Plot for artificially truncated spacer deletion, left thresholded
print(data[3][0])
x, y = startend_leftthresh(delspacertruncate, Div2, 10)
plotstartend(x, y, Div2)

RUN144_298_DELSPACER_DIV


In [38]:
#Plot for artificially truncated 3' homology deletion, left thresholded
print(data[2][0])
x, y = startend_leftthresh(del3homtruncate, Div2, 10)
plotstartend(x, y, Div2)

RUN144_296_DEL3HOM_DIV
